In [ ]:
!pip install -U pandas==0.25.0

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 102kB 4.3MB/s 
     |████████████████████████████████| 286kB 25.9MB/s 
     |████████████████████████████████| 10.4MB 27.4MB/s 
     |████████████████████████████████| 317kB 46.4MB/s 
     |████████████████████████████████| 6.7MB 51.0MB/s 
     |████████████████████████████████| 748.9MB 23kB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
     |████████████████████████████████| 102kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
     |████████████████████████████████| 7.3MB 38.5MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp37-none-any.whl size=47640 sha256=3d736b1f2e95f4e35cb6619f037c3e6d9e36e9d2f2fca963035f01ed1aa72115
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e

In [1]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
download_model(model='bert-squad_1.1', dir='./models')


bert_qa.joblib already downloaded


In [3]:
mkdir -p ./data/amazon/

In [5]:
ls data/amazon

Amazon-Q4-2020-Earnings-Release.pdf  AMZN-Q3-2020-Earnings-Release.pdf
AMZN-Q1-2020-Earnings-Release.pdf    Q2-2020-Amazon-Earnings-Release.pdf


In [9]:
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf
!wget -P ./data/amazon/ https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf

--2021-03-27 09:44:53--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 167.114.174.191
Connecting to s2.q4cdn.com (s2.q4cdn.com)|167.114.174.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 869995 (850K) [application/pdf]
Saving to: ‘./data/amazon/Amazon-Q4-2020-Earnings-Release.pdf’

Amazon-Q4-2020-Earn 100%[===================>] 849.60K   888KB/s    in 1.0s    

2021-03-27 09:44:55 (888 KB/s) - ‘./data/amazon/Amazon-Q4-2020-Earnings-Release.pdf’ saved [869995/869995]

--2021-03-27 09:44:55--  https://s2.q4cdn.com/299287126/files/doc_financials/2020/q3/AMZN-Q3-2020-Earnings-Release.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 167.114.174.191
Connecting to s2.q4cdn.com (s2.q4cdn.com)|167.114.174.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 945573 (923K) [application/pdf]
Saving to: ‘./data/amazon/AMZN-Q3-2020-Earnings-Release.pdf’

AMZ

In [6]:
#specify data path
df = pdf_converter(directory_path='./data/amazon/')
df.head()

,title,paragraphs
0,Q2-2020-Amazon-Earnings-Release,[AMZN-2020.06.30-EX99.1 AMAZON.COM ANNOUNCES S...
1,Amazon-Q4-2020-Earnings-Release,[Microsoft Word - AMZN-2020.12.31-EX99.1_02022...
2,AMZN-Q3-2020-Earnings-Release,[Microsoft Word - AMZN-Q3-2020-Earnings-Releas...
3,AMZN-Q1-2020-Earnings-Release,[Microsoft Word - Earnings Release_4.30.20_12P...


In [7]:
#Use Pre Defined Readering Models to train the data prediction
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 322812.32B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [11]:
#Question 
query = 'how was third quarter net sales?'
prediction = cdqa_pipeline.predict(query,4)
#following prints the prediction based on question asked. stating what is the answer and where it found it.
# print('query: {}'.format(query))
# print('answer: {}'.format(prediction[0]))
# print('title: {}'.format(prediction[1]))
# print('paragraph: {}'.format(prediction[2]))

In [12]:
prediction

[('increased 27%',
  'AMZN-Q1-2020-Earnings-Release',
  'SEATTLE—(BUSINESS WIRE) April 30, 2020—Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its first quarter ended March 31, 2020.  •  Operating cash flow increased 16% to $39.7 billion for the trailing twelve months, compared with $34.4 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow increased to $24.3 billion for the trailing twelve months, compared with $23.0 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow less principal repayments of finance leases and financing obligations decreased to $14.3 billion for the trailing twelve months, compared with $15.1 billion for the trailing twelve months ended March 31, 2019. •  Free cash flow less equipment finance leases and principal repayments of all other finance leases and financing obligations decreased to $11.7 billion for the trailing twelve months, compared with $11.8 billion for the trailing twelve mo